Antonio Merino Gallardo

# Criptosistema de McEliece

Presentemos en este cuaderno un ejemplo de cifrado y descifrado con el criptosistema de McEliece. Para ello, haremos uso de la clase CodigoGoppaBinario implementada en el archivo *codigo_goppa_binario.py*.

Importamos la clase.

In [1]:
from codigo_goppa_binario import CodigoGoppaBinario

Determinamos el grado $t$ del polinomio de Goppa y el cuerpo finito $\mathbb{F}_{2^m}$ sobre el que trabajar.

In [2]:
t = 3
m = 4
n = 2^m
F= GF(2^m)
R.<x> = F[]

Indicamos semilla para la reproducibilidad de los cálculos aleatorios que realizaremos.

In [3]:
set_random_seed(0)

Generamos aleatoriamente un polinomio irreducible de grado $t$ sobre $\mathbb{F}_{2^m}$ que será el polinomio de Goppa $g$.

In [4]:
g = R(0)
while not g.is_irreducible():
    g = R.random_element(t)
g = g/g.list()[len(g.list())-1]
g

x^3 + (z4^3 + 1)*x^2 + (z4^3 + 1)*x + z4^3 + z4^2 + z4 + 1

Tomamos como conjunto de definición $L=\mathbb{F}_{2^m}$ y construimos el código Goppa binario irreducible.

In [5]:
L = F.list()
C = CodigoGoppaBinario(g, L)
C

[16, 4] Código Goppa Binario

Tomamos $G$ una matriz generadora del código.

In [6]:
G = C.matriz_generadora(); G

[1 0 0 1 0 1 1 0 0 0 0 1 0 0 1 1]
[0 1 0 1 0 0 0 0 1 1 1 1 0 1 1 1]
[0 0 1 0 0 1 0 0 0 0 1 1 1 1 0 1]
[0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 1]

Llamamos $k$ a la dimensión del código.

In [7]:
k = C.dimension(); k

4

Generamos $S$ una matriz regular aleatoria de dimensión $k \times k$.

In [8]:
S = random_matrix(GF(2), k)
while S.is_singular():
    S = random_matrix(GF(2), k)
S

[1 0 1 1]
[1 1 1 1]
[0 0 1 0]
[0 1 1 1]

Generamos $P$ una matriz de permutaciones aleatoria de dimensión $n \times n$.

In [9]:
P = Permutations(n).random_element().to_matrix()
P = matrix(GF(2), P)
P

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]

Construimos la matriz generadora $G_2=SGP$, que constituirá la clave pública junto con el valor $t$.

In [10]:
G2 = S*G*P; G2

[1 0 0 1 1 0 1 0 0 1 1 1 1 1 1 0]
[1 0 0 0 1 1 0 1 1 0 1 0 1 0 1 1]
[0 1 1 0 1 1 0 0 1 0 0 1 0 0 0 1]
[0 1 0 0 1 1 0 1 0 1 1 1 1 1 0 1]

### Cifrado

Generemos una palabra aleatoria $u \in \mathbb{F}_{2}^k$ para cifrarla.

In [11]:
u = VectorSpace(GF(2),k).random_element(); u

(0, 1, 1, 1)

Para cifrar, generamos un patrón de error $e$ aleatorio de peso $t$.

In [12]:
errores = Combinations(range(n),t).random_element(); errores

[2, 5, 14]

In [13]:
e = vector(GF(2), n*[0])
for pos in errores:
    e[pos]=1
e

(0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0)

Ciframos $u$ multiplicando por la matriz generadora $G_2$ y sumándole el patrón de error.

In [14]:
x = u*G2 + e; x

(1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1)

### Descifrado

Procedamos ahora a descifrar la palabra $x \in \mathbb{F}_{2}^n$.

Deshacemos la permutación de las columnas multiplicando por la inversa de $P$.

In [15]:
x2 = x*P.inverse(); x2

(1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0)

Aplicamos el algoritmo de Patterson para corregir los $t$ errores.

In [16]:
x3 = C.decodificar(x2); x3

(1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0)

Resolvemos el sistema lineal para obtener una palabra $u_2 \in \mathbb{F}_2^k$.

In [17]:
u2 = G.solve_left(x3); u2

(1, 0, 1, 0)

Finalmente, deshacemos el cambio de base producido por $S$.

In [18]:
u3 = u2*S.inverse(); u3

(0, 1, 1, 1)

Comprobamos que hemos descifrado correctamente.

In [19]:
u3 == u

True